# Ćwiczenia 6. Walidacja Krzyżowa

## PyTorch na następne ćwiczenia.

Proszę zainstalować pakiet [PyTorch](https://pytorch.org/) oraz torchvision na kolejne zajęcia. Jeśli używane, mając swoje środowisko aktywne użyć:

 * GPU: `conda install pytorch torchvision cudatoolkit=9.0 -c pytorch`
 * tylko CPU: `conda install pytorch torchvision cpuonly  -c pytorch`

## Klasyfikacja

Dzisiaj na zajęciach zajmiemy się problemem klasyfikacji. Podobnie do regresji liniowej jest to przykład uczenia nadzorowanego, ale zamiast przewidywać konkretną liczbę dla danej obserwacji, przewidujemy jego przynajeżność do jednej z *k* klas. Na tych zajęciach będziemy rozważać klasyfikacje binarną, czyli uczyć modele odpowiadające funkcji:

$$ f(x) = y, \quad y \in \{0,1\} $$

Poniżej ładowane są dane, do razu już podzielone na dwie części.

In [1]:
import numpy as np
from utils import get_data

X_train, X_test, y_train, y_test = get_data()

## Zadanie 1.1 (0.5 pkt.)

Używając modelu [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) z pakietu sklearn uzyskać 100% dokładność (mierzoną miarą [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html))na zbiorze treningowym. Państwa zadanie polega na dobraniu parametru `gamma`, dla ułatwienia proszę nie zmieniać pozostałych domyślnych parametrów. Zalecany przedział parametru `gamma` to wartości z przedziału [0, 1] w skali logarytmicznej.

In [2]:
from sklearn.svm import SVC

gamma = 1

svm = SVC(gamma=gamma)
svm.fit(X_train, y_train)
svm.score(X_train, y_train)

1.0

In [3]:
# test
best_gamma = 1

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
train_acc = svm.score(X_train, y_train)

assert train_acc == 1.

## Zadanie 1.2 (0.5 pkt.)
Używając tej samej rodziny modeli znajdź tym razem taką wartość `gamma`, która daje najlepszy wynik na zbiorze **testowym**. Powinieneś(-aś) być w stanie osiągnąć wynik co najmniej `0.95` accuracy. 

In [4]:
np.logspace(-10, 1).shape

(50,)

In [5]:
from sklearn.svm import SVC

for gamma in np.logspace(-10, 1):
    svm = SVC(gamma=gamma)
    svm.fit(X_train, y_train)
    print(f'gamma: {gamma}, score: {svm.score(X_test, y_test)}')

gamma: 1e-10, score: 0.6223776223776224
gamma: 1.67683293681101e-10, score: 0.6223776223776224
gamma: 2.811768697974225e-10, score: 0.6223776223776224
gamma: 4.71486636345739e-10, score: 0.6223776223776224
gamma: 7.906043210907701e-10, score: 0.6223776223776224
gamma: 1.3257113655901108e-09, score: 0.6293706293706294
gamma: 2.222996482526191e-09, score: 0.6923076923076923
gamma: 3.727593720314938e-09, score: 0.7692307692307693
gamma: 6.250551925273976e-09, score: 0.8251748251748252
gamma: 1.0481131341546852e-08, score: 0.8671328671328671
gamma: 1.7575106248547893e-08, score: 0.8881118881118881
gamma: 2.9470517025518096e-08, score: 0.8881118881118881
gamma: 4.9417133613238385e-08, score: 0.9230769230769231
gamma: 8.286427728546843e-08, score: 0.9370629370629371
gamma: 1.389495494373136e-07, score: 0.9370629370629371
gamma: 2.329951810515372e-07, score: 0.9370629370629371
gamma: 3.906939937054621e-07, score: 0.9440559440559441
gamma: 6.55128556859551e-07, score: 0.951048951048951
gamma: 

In [6]:
# test
best_gamma = 6.866488450042999e-05

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
test_acc = svm.score(X_test, y_test)

assert test_acc >= 0.95

### Problem.

**W poprzednim zadaniu zakładaliśmy, że podział na zbiór trenujący/testujący jest nam podany z góry, ale co jeśli go nie mamy?**

Możemy oczywiście sami arbitralnie wybrać część datasetu i uznać ją za nasz zbiór testowy, ale to mogą się z tym wiązać dodatkowe problemy: co jeśli wybrany przez nas fragment jest akurat różny od reszty datasetu, lub odwrotnie?

**Rozwiązanie:** Walidacja Krzyżowa.

1. Podziel dataset na zadaną przez parametr `k` liczbę (prawie) równych grup.
2. Dla każdego podziału, zwróć jedną z tych części jako zbiór testowy, a sumę reszty jako zbiór treningowy.
3. Po nauczeniu łącznie `k` modeli, uśrednij ich wyniki na zbiorach testowych i zwróć jako ostateczny wynik.

## Zadanie 2. (2 pkt.)

Państwa zadaniem jest zaimplementowanie walidacji krzyżowej, czyli funkcji, która dla podanego datasetu w postaci macierzy danych `X` i wektora etykiet `y` zwróci listę `k` krotek: 
  
  `(treningowe_dane, treningowe_etykiety, testowe_dane, testowe_etykiety)`
  
podziałów dokonanych przez walidację krzyżową. Następnie należy użyć modelu z poprzedniego zadania dla policzenia dokładności na zbiorze testowym dla walidacji krzyżowej.

Proszę **nie** korzystać z gotowych rozwiązań dostępnych w pakiecie sklearn.


In [7]:
from typing import List, Tuple


def cross_validation(X: np.ndarray, y: np.ndarray, k: int) -> List[Tuple[np.ndarray, np.ndarray, 
                                                                         np.ndarray, np.ndarray]]:
    
    res = []
    N = len(X)
    
    fold_len = np.full(k, N // k, dtype=np.int)
    fold_len[:N % k] += 1
    
    for i in range(k):
        X_train, y_train = [], []
        current = 0
        for j, length in enumerate(fold_len):
            start, stop = current, current + length
            if j == i:
                X_test = X[start:stop]
                y_test = y[start:stop]
            else:
                X_train += list(X[start:stop])
                y_train += list(y[start:stop])
            current = stop
        res.append((np.array(X_train), np.array(y_train), X_test, y_test))
    return res

In [8]:
from checker import test_cv

test_cv(cross_validation)

In [9]:
X, y = get_data(split=False)

acc = []
for X_train, y_train, X_test, y_test in cross_validation(X, y, k=5):
    acc.append(svm.fit(X_train, y_train).score(X_test, y_test))
    
cv_accuracy = np.asarray(acc).mean()
cv_accuracy

0.924437199192672

## Zadanie 3 (1 pkt.)

Mając już lepszą metodę walidacji naszego rozwiązania Państwa zadaniem jest znalezienia najlepszego zestawu hiperparametrów dla modelu z poprzednich zadań, lecz tym razem będziemy szukać również parametru `C`. Parametru C zaleca się szukać w przedziale $(0, + \infty)$ również w skali logarytmicznej.

W zadaniu należy oczywiście skorzystać z zaimplementowanej przez Państwa walidacji krzyżowej. Ponieważ dla dwóch parametrów `C` oraz `gamma` możliwych kombinacji do przetestowania robi są dość sporo dla przetestowania dużych zakresów zalecane są również inne metody przeszukiwania, takie jak:

* przeszukiwanie po kolei z jednym z parametrów ustalonym na stałą.
* przeszukiwanie losowe obu parametrów

Oczywiście jeśli zasoby na to pozwalają można szukać tzw. "grid searchem".

Powinno udać się Państwu wyciągnąć przynajmniej `0.94` accuracy na walidacji krzyżowej.

In [10]:
from sklearn.svm import SVC
from tqdm import tqdm

X, y = get_data(split=False)

best_gamma, best_C, best_acc = 0, 0, 0
for gamma in tqdm(np.logspace(-10, 1)):
    for C in np.logspace(-10, 3):
        acc = []
        for X_train, y_train, X_test, y_test in cross_validation(X, y, k=5):
            svm = SVC(C=C, gamma=gamma)
            acc.append(svm.fit(X_train, y_train).score(X_test, y_test))
        cv_accuracy = np.asarray(acc).mean()
        if cv_accuracy > best_acc:
            best_gamma = gamma
            best_C = C
            best_acc = cv_accuracy
print(best_gamma, best_C, best_acc)

100%|██████████| 50/50 [02:34<00:00,  3.08s/it]

5.179474679231212e-06 1000.0 0.9542928116752059


## Zadanie 4. (3 punkty)

Załóżmy, że naszym problemem jest zdecydowanie, która rodzina modeli *SVM* najlepiej radzi sobei z naszym datasetem. Przez rodzinę rozumiemy tutaj modele SVM z różną *funkcją bazwoą* (zwaną często *funkcją jądra*). W pakiecie mamy dostępne kilka możliwości, włącznie z podawaniem swoich własnych, ale w tym zadaniu skupimy się na czterech dostępnych od ręki: `linear`, `poly`, `rbf`, `sigmoid`.

Wiemy jak znaleźć najlepszy zestaw parametrów dla danej rodziny modeli, zrobiliśmy to do tej pory dla domyślnej funkcji bazowej czyli `rbf`. Jak jednak mamy "uczciwie" porównać wyniki modeli pomiędzy sobą? Do tej pory patrzyliśmy na wyniki modelu dla datasetu testowego i to na podstawie tego wyniku wybieraliśmy najlepsze hiperparametry. Jakie mogą być z tym problemy? Overfitting?

Rozwiązanie: jeszcze jedna walidacja krzyżowa!

1. Pierwsza walidacja krzyżowa podzieli nam nasz zbiór na treningowy i testowy. Te testowe zbiory będą naszymi ostatecznymi zbiorami testowymi, na których nie będziemy w żaden sposób się uczyć czy szukać hiperparametrów. 
2. Następnie nasz zbiór treningowy podzielimy ponownie walidacją krzyżową na dwie części: faktyczny treningowy i walidacyjny. Tych dwóch podziałów będziemy używać jak poprzednio do uczenia modelu i testowania hiperparametrów.
3. Po znalezieniu najlepszego zestawu hiperparametrów nauczymy ostatecznie nasz model na sumie zbiorów treningowego i walidacyjnego i sprawdzimy jego dokładność na ostatecznym zbiorze testowym.


**Uwaga**: parametr `C` używany jest dla każdej możliwej funkcji bazowej. Proszę sprawdzić jakie parametry są używane dla jakich funkcji jądra. 
**Hint**: parametry, które mogą państwa interesować to oczywiście `kernel`, oraz `C`, `degree`, `gamma`, `coef0`.

In [ ]:
from sklearn.svm import SVC
from tqdm import tqdm

X, y = get_data(split=False)

test_acc = []
for X_train_outer, y_train_outer, X_test_outer, y_test_outer in cross_validation(X, y, k=3):
    best_kernel, best_C, best_acc = '', 0, 0
    for C in tqdm(np.logspace(1, 10, num=10)):
        for kernel in tqdm(['linear', 'poly', 'rbf', 'sigmoid']):
            acc = []
            for X_train, y_train, X_test, y_test in tqdm(cross_validation(X_train_outer, y_train_outer, k=3)):
                svm = SVC(C=C, kernel=kernel, gamma='scale')
                acc.append(svm.fit(X_train, y_train).score(X_test, y_test))
            cv_accuracy = np.asarray(acc).mean()
            if cv_accuracy > best_acc:
                best_C = C
                best_acc = cv_accuracy
                best_kernel = kernel
    svm = SVC(C=best_C, kernel=best_kernel, gamma='scale')
    test_acc.append(svm.fit(X_train_outer, y_train_outer).score(X_test_outer, y_test_outer))
print(np.asarray(test_acc).mean())




  0%|          | 0/2 [00:00<?, ?it/s]



  0%|          | 0/4 [00:00<?, ?it/s]




  0%|          | 0/3 [00:00<?, ?it/s]




 33%|███▎      | 1/3 [00:05<00:10,  5.29s/it]




 67%|██████▋   | 2/3 [00:12<00:06,  6.01s/it]




100%|██████████| 3/3 [00:17<00:00,  5.72s/it]




 25%|██▌       | 1/4 [00:17<00:51, 17.18s/it]




100%|██████████| 3/3 [00:00<00:00, 111.74it/s]





100%|██████████| 3/3 [00:00<00:00, 126.95it/s]





100%|██████████| 3/3 [00:00<00:00, 77.07it/s]A




100%|██████████| 4/4 [00:17<00:00,  4.33s/it]



 50%|█████     | 1/2 [00:17<00:17, 17.33s/it]



  0%|          | 0/4 [00:00<?, ?it/s]




  0%|          | 0/3 [00:00<?, ?it/s]